In [1]:
#!pip install torch


In [67]:
#packages 
#Load packages :pytorch, sklearn etc.,
import numpy as np
import pandas as pd
import random
import os

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from tqdm.notebook import trange, tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import copy
import time

In [10]:
#reading in the .npz file to train the model 
def get_training_data(load=False, datafile ='test.npz'):
    
    if not load:
        # This grabs the training data files from the appropriate directory
        mypath = r'C:\Users\Sam\Documents\PSU\DS 440\DS440-CAPSTONE-main\test.npz'  #change path to fit your path file
        training_data_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
        
        training_t = []  
        training_f = []
        training_class = []
        training_info = []
        for f in training_data_files:
            data = np.load(mypath+f)
            all_time=data['all_time']
            all_flux=data['all_flux']
            ra = data['ra']
            dec = data['dec']
            injmu_percentile=data['injmu_percentile']
            injduration=data['injduration']
            injloc_x=-1 * data['injloc_x']
            injloc_y=-1 * data['injloc_y']
            injpeak=data['injpeak']
            
            info = ra,dec,injmu_percentile,injduration,injloc_x,injloc_y,injpeak #injduration, injpeak
            for i, t in enumerate(all_time):
                f = all_flux[i]
                row = np.floor(i/9) - 4
                col = i%9 - 4
                if np.all(np.isnan(f)):
                    continue
                if np.sqrt((row - injloc_x)**2 + (col-injloc_y)**2)<=2:
                    training_t.append(t)
                    training_f.append(f)
                    training_class.append(1)
                    training_info.append(info)
                    
                elif np.sqrt((row - injloc_x)**2 + (col-injloc_y)**2)>5:
                    training_t.append(t)
                    training_f.append(f)
                    training_class.append(0)
                    training_info.append(info)
        np.savez(datafile,training_t = training_t, training_f = training_f,
                training_class = training_class, training_info = training_info)
        return training_t, training_f,training_class,training_info
    else:
        data = np.load(datafile, allow_pickle=True)
        training_t = data['training_t']         # needs to be padded and then shuffled
        training_f = data['training_f']         
        training_class = data['training_class']  
        training_info = data['training_info']   
        return training_t, training_f,training_class,training_info
training_t, training_f,training_class,training_info = get_training_data(load=True)

In [11]:
#padding the arrays to match the max size
longest_len = 3658
for i in np.arange(len(training_t)):
    mylen = len(training_t[i])
    if mylen>longest_len:
        longest_len = mylen
new_training_t = np.zeros((len(training_t),longest_len))
new_training_f = np.zeros((len(training_t),longest_len))
for i in np.arange(len(training_t)):
    nl = len(training_t[i])
    before_N=int(longest_len/2-(nl/2))
    new_training_t[i,before_N:nl+before_N] = training_t[i]   # Data to be used
    new_training_f[i,before_N:nl+before_N] = training_f[i]   # Data to be used

In [12]:
type(new_training_f)

numpy.ndarray

In [13]:
new_training_f

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [25]:
new_training_f.shape

(12116, 3658)

In [26]:
training_class.shape

(12116,)

In [41]:
class CustomDataset(data.Dataset):
    def __init__(self, training_class, new_training_f, transform=None, target_transform=None):
        self.labels = training_class
        self.data = new_training_f
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        #img_path = os.path.join(self.data, self.labels.iloc[idx, 0])
        curve = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            curve = self.transform(curve)
        if self.target_transform:
            label = self.target_transform(label)
        return curve, label

In [42]:
train = CustomDataset(training_class, new_training_f, transform=None, target_transform=None)

In [47]:
VALID_RATIO = 0.9
TEST_RATIO = 0.66

n_train_examples = int(len(train) * TEST_RATIO)
n_test_examples = len(train) - n_train_examples
n_train_examples = int(n_train_examples * VALID_RATIO)
n_valid_examples = len(train) - n_test_examples - n_train_examples

In [48]:
train_data, test_data, valid_data = data.random_split(train, [n_train_examples, n_test_examples, n_valid_examples])

In [49]:
BATCH_SIZE = 64

train_iterator = data.DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data, batch_size=BATCH_SIZE)

test_iterator = data.DataLoader(test_data, batch_size=BATCH_SIZE)

In [54]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.input_fc = nn.Linear(input_dim, 1400)
        self.hidden_fc1 = nn.Linear(1400, 530)
        self.hidden_fc2 = nn.Linear(530, 200)
        self.hidden_fc3 = nn.Linear(200, 80)
        self.output_fc = nn.Linear(80, output_dim)

    def forward(self, x):

        # x = [batch size, height, width]

        batch_size = x.shape[0]

        x = x.view(batch_size, -1)

        # x = [batch size, height * width]

        h_1 = F.relu(self.input_fc(x))

        # h_1 = [batch size, 1400]

        h_2 = F.relu(self.hidden_fc1(h_1))

        # h_2 = [batch size, 530]
        
        h_3 = F.relu(self.hidden_fc2(h_2))

        # h_3 = [batch size, 200]
        
        h_4 = F.relu(self.hidden_fc3(h_3))

        # h_4 = [batch size, 80]

        y_pred = self.output_fc(h_4)

        # y_pred = [batch size, output dim]

        return y_pred, h_4

define our model by creating an instance of it and setting the correct input and output dimensions.

In [55]:
INPUT_DIM = 3658
OUTPUT_DIM = 2

model = MLP(INPUT_DIM, OUTPUT_DIM)

calculate the number of trainable parameters

In [56]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

The first layer has 3658 neurons connected to 1400 neurons, so 3658*1400 weighted connections plus 1400 bias terms.

The second layer has 1400 neurons connected to 530 neurons, 1400*530 weighted connections plus 530 bias terms.

The third layer has 530 neurons connected to 200 neurons, 530*200 weighted connections plus 200 bias terms.

The fourth layer has 200 neurons connected to 80 neurons, 200*80 weighted connections plus 80 bias terms.

The fifth layer has 80 neurons connected to 2 neurons, 80*2 weighted connections plus 2 bias terms.

$$3658 \cdot 1400 + 1400 + 1400 \cdot 530 + 530 + 530 \cdot 200 + 200 + 200 \cdot 80 + 80 + 80 \cdot 2 + 2 = 5,987,572 $$

In [57]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,987,572 trainable parameters


In [58]:
optimizer = optim.Adam(model.parameters())

In [59]:
criterion = nn.CrossEntropyLoss()

In [60]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [61]:
model = model.to(device)
criterion = criterion.to(device)

In [62]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [63]:
def train(model, iterator, optimizer, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for (x, y) in tqdm(iterator, desc="Training", leave=False):

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        y_pred, _ = model(x)

        loss = criterion(y_pred, y)

        acc = calculate_accuracy(y_pred, y)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [64]:
def evaluate(model, iterator, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for (x, y) in tqdm(iterator, desc="Evaluating", leave=False):

            x = x.to(device)
            y = y.to(device)

            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [65]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [68]:
EPOCHS = 10

best_valid_loss = float('inf')

for epoch in trange(EPOCHS):

    start_time = time.monotonic()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

  0%|          | 0/10 [00:00<?, ?it/s]

Training:   0%|          | 0/113 [00:00<?, ?it/s]

RuntimeError: expected scalar type Float but found Double

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)